<a href="https://colab.research.google.com/github/if001/redpajama-lora-instruct-ja/blob/main/bolly_ja_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setup


In [ ]:
%cd /content
!git clone https://github.com/if001/redpajama-lora-instruct-ja.git
repo_dir='/content/redpajama-lora-instruct-ja'
%cd $repo_dir

!pip install -r requirements.txt

/content
Cloning into 'redpajama-lora-instruct-ja'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 102 (delta 46), reused 82 (delta 28), pack-reused 0
Receiving objects: 100% (102/102), 20.87 MiB | 17.36 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/redpajama-lora-instruct-ja
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-53jjw2nt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-53jjw2nt
  Resolved https://github.com/huggingface/peft.git to commit 0fcc30dd43a23081018570fc7a76e4b3cf282c23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.9

In [ ]:
#@title Google Drive Options { display-mode: "form" }
save_models_to_drive = True #@param {type:"boolean"}
drive_mount = '/content/drive' #@param {type:"string"}

# output_dir = 'rinna_dolly_3B' #@param {type:"string"}
# output_dir = 'calm_dolly_ja' #@param {type:"string"}
# output_dir = 'rinna_dolly_3B_rinna_format' #@param {type:"string"}
output_dir = 'calm_dolly_7B' #@param {type:"string"}


import os
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    output_path = f"{drive_mount}/MyDrive/models/{output_dir}" if save_models_to_drive else f"/content/{output_dir}"
else:
    output_path = "/content"


print(f"Saving models to {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving models to /content/drive/MyDrive/models/calm_dolly_7B


## train

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%cd $repo_dir
!git pull
# !pip install -r requirements.txt

/content/redpajama-lora-instruct-ja
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 588 bytes | 588.00 KiB/s, done.
From https://github.com/if001/redpajama-lora-instruct-ja
   7df09a6..5fe6036  main       -> origin/main
Updating 7df09a6..5fe6036
Fast-forward
 templates/dolly_ja_rinna.json | 6 ++++++
 1 file changed, 6 insertions(+)
 create mode 100644 templates/dolly_ja_rinna.json


In [ ]:
%cd $repo_dir

# base_model="togethercomputer/RedPajama-INCITE-Base-3B-v1"
# base_model="togethercomputer/RedPajama-INCITE-Base-7B-v0.1"
# base_model="cyberagent/open-calm-3b"
base_model="rinna/japanese-gpt-neox-3.6b-instruction-sft"
data_path="kunishou/databricks-dolly-15k-ja"
template = 'dolly_ja'
template = 'dolly_ja_rinna'


!python3 finetune.py \
--base_model=$base_model \
--batch_size=128 \
--micro_batch_size=2 \
--prompt_template_name=$template \
--cutoff_len=1024 \
--output_dir=$output_path \
--num_epochs=2 \
--data_path=$data_path \
--gradient_checkpointing=False \
--resume_from_checkpoint=$output_path

/content/redpajama-lora-instruct-ja
2023-05-21 04:39:22.180603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-chi

## row infarence

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft")

#if torch.cuda.is_available():
#    model = model.to("cuda")

In [ ]:
def create_prompt(qa):
  PROMPT_FORMAT = '以下は、ユーザーの入力です。適切な応答を出力してください。<NL><NL>ユーザー: {instruction}<NL><NL>システム: <NL>'
  PROMPT_FORMAT = '以下は、ある作業を記述した指示です。依頼を適切に完了させる応答を書きなさい\n\n### 指示:\n{instruction}\n\n### 応答:\n'
  prompt = PROMPT_FORMAT.format(instruction=qa)
  return prompt
  # return f"ユーザー: {qa}<NL>システム:"

def gen(qa):
  prompt = create_prompt(qa)
  print(prompt)
  token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  token_ids = token_ids.to(model.device)

  with torch.no_grad():
    output_ids = model.generate(
          token_ids,
          do_sample=True,
          max_new_tokens=128,
          temperature=0.9,
          pad_token_id=tokenizer.pad_token_id,
          bos_token_id=tokenizer.bos_token_id,
          eos_token_id=tokenizer.eos_token_id
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
  # output = output.replace("<NL>", "\n")
  print(output)

In [ ]:
gen('日本で一番高い山を教えてください。')

以下は、ある作業を記述した指示です。依頼を適切に完了させる応答を書きなさい

### 指示:
日本で一番高い山を教えてください。

### 応答:

北から順番に並べると、次のようになります: 9,8と称された峰から、最高峰の槍ヶ岳と次に高い山は御嶽山です。 この峰は、海抜21,003フィートの日本で2番目に高い山です。 したがって、答えは御嶽山です。</s>


In [ ]:
gen('色の三原色を教えてください。')

以下は、ある作業を記述した指示です。依頼を適切に完了させる応答を書きなさい

### 指示:
色の三原色を教えてください。

### 応答:

はい、こちらになります。</s>


## infarence


In [ ]:
import sys
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

from peft import PeftModel


lora_weights = output_path
# lora_weights = '/content/drive/MyDrive/models/rinna_dolly_3B'
lora_weights = '/content/drive/MyDrive/models/rinna_dolly_3B_rinna_format'


base_model = "togethercomputer/RedPajama-INCITE-Base-3B-v1"
base_model="togethercomputer/RedPajama-INCITE-Base-7B-v0.1"
base_model="cyberagent/open-calm-3b"
base_model="rinna/japanese-gpt-neox-3.6b-instruction-sft"

tokenizer = AutoTokenizer.from_pretrained(base_model)

# device = None
device = "cuda"

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
                base_model,
                load_in_8bit=True,
                torch_dtype=torch.float16,
                device_map="auto",
            )
    model = PeftModel.from_pretrained(
            model,
            lora_weights,
            torch_dtype=torch.float16,
        )
else:
    model = AutoModelForCausalLM.from_pretrained(
            base_model, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
            model,
            lora_weights,
            device_map={"": device},
        )
    
model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
# tokenizer.padding_side = "left"  # Allow batched inference


def gen_prompt(inp):
    # PROMPT_FORMAT = '以下は、あるタスクを記述した指示です。質問に対する適切な回答を書きなさい。\n\n### 指示:\n{instruction}\n\n### 応答:\n'
    PROMPT_FORMAT = '以下は、ユーザーの入力です。適切な応答を出力してください。<NL><NL>ユーザー: {instruction}<NL><NL>システム: <NL>'
    prompt = PROMPT_FORMAT.format(instruction=inp)
    return prompt


def evaluate(
        instruction,
        input=None,
        temperature=0.7,
        top_p=0.8,
        top_k=40,
        max_new_tokens=128,
        stream_output=False,
        **kwargs,
    ):
    # tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
    # prompt = prompter.generate_prompt(instruction, input)
    prompt = gen_prompt(instruction)
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True,
            **kwargs,
        )
       
    with torch.no_grad():
        #generation_output = model.generate(
        #        input_ids=input_ids,
        #        generation_config=generation_config,
        #        return_dict_in_generate=True,
        #        output_scores=True,
        #        max_new_tokens=max_new_tokens,
        #)
        generation_output = model.generate(
                input_ids=input_ids,
                do_sample=True,
                temperature=0.9,
                pad_token_id=tokenizer.pad_token_id,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
        )
        s = generation_output.sequences[0]
        output = tokenizer.decode(s)
    return output

In [ ]:
evaluate('色の三原色を教えてください')

'以下は、ユーザーの入力です。適切な応答を出力してください。<NL><NL>ユーザー: 色の三原色を教えてください<NL><NL>システム: <NL><NL>「色の三原色」というのは、赤・青・黄色のことです。この3つの色は、通常、赤と青と黄色を混ぜると緑色になります。しかし、3色を混ぜると、黒色になります。これは、光合成が光が吸収される過程と、光が電子を励起する過程の両方を記述しているためです。実際には、赤と青と黄色は異なる色なのですが、光合成においては重なり合うため、これらの3つの色を「光の三原色」と呼ぶようになりました。つまり、赤と青と黄色は、光の3つの色と考えることができます'

In [ ]:
evaluate('日本で最も高い山を教えてください')

'以下は、ユーザーの入力です。適切な応答を出力してください。<NL><NL>ユーザー: 日本で最も高い山を教えてください<NL><NL>システム: <NL><NL>「日本で最も高い」とされているものは、確かに約2,500メートル (9,100 ft)の高さです。他の最も高い山は、標高4,850メートル (13,750 ft)のエベレスト山です。以下は、主要な日本の標高を持つ山岳のリストです:<NL><NL>・   御嶽山<NL>・   奥穂高岳<NL>・   槍ヶ岳<NL>・   剱岳<NL>・   乗鞍岳<NL>・   白山<NL'

In [ ]:
evaluate('日本で２番めに高い山を教えてください')

'以下は、ユーザーの入力です。適切な応答を出力してください。<NL><NL>ユーザー: 日本で2番めに高い山を教えてください<NL><NL>システム: <NL>日本で2番目に高い山は、槍ヶ岳(3,810m)です。</s>'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft")

for v in model.state_dict().items():
  print(v[0])

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

### cpp (fail)

In [ ]:
%cd /content
#!git clone https://github.com/ggerganov/llama.cpp.git
#cpp_repo="/content/llama.cpp"

!git clone https://github.com/ggerganov/ggml
cpp_repo="/content/ggml"


/content
Cloning into 'ggml'...
remote: Enumerating objects: 1512, done.
remote: Counting objects: 100% (639/639), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 1512 (delta 540), reused 553 (delta 489), pack-reused 873
Receiving objects: 100% (1512/1512), 3.94 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (982/982), done.


In [ ]:
%cd $cpp_repo
!mkdir -p build
%cd $cpp_repo/build
!cmake ..
# !cmake --build . --config Release
!make -j


/content/ggml
/content/ggml/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.25.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Linux detected
-- x86 detected
-- Linux detected
--

In [ ]:
!mkdir -p $cpp_repo/models/3B
%cd $cpp_repo/models/3B

!wget "https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin"

/content/llama.cpp/models/3B
--2023-05-17 09:39:17--  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 13.32.121.122, 13.32.121.103, 13.32.121.37, ...
Connecting to huggingface.co (huggingface.co)|13.32.121.122|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/e9/58e9ff5c59adafdd9012d69c25c4272a64be26c511f2c3f69fe8a17881ac2120/0c6124c628f8ecc29be1b6ee0625670062340f5b99cfe543ccf049fa90e6207b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1684574501&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2U5LzU4ZTlmZjVjNTlhZGFmZGQ5MDEyZDY5YzI1YzQyNzJhNjRiZTI2YzUxMWYyYzNmNjlmZThhMTc4ODFhYzIxMjAvMGM2MTI0YzYyOGY4ZWNjMjliZTFiNmVlMDYyNTY3MDA2MjM0MGY1Yjk5Y2ZlNT

In [ ]:
!mkdir -p $cpp_repo/models
%cd $cpp_repo/models

!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.model
!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.vocab
!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/config.json

/content/ggml/models
--2023-05-17 14:54:53--  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.model
Resolving huggingface.co (huggingface.co)... 13.35.24.18, 13.35.24.126, 13.35.24.127, ...
Connecting to huggingface.co (huggingface.co)|13.35.24.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/e9/58e9ff5c59adafdd9012d69c25c4272a64be26c511f2c3f69fe8a17881ac2120/7d78ab344146700112cd41628ac7ce54b79c0868fe0c7c201750d8237b54dbb4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27spiece.model%3B+filename%3D%22spiece.model%22%3B&Expires=1684592612&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2U5LzU4ZTlmZjVjNTlhZGFmZGQ5MDEyZDY5YzI1YzQyNzJhNjRiZTI2YzUxMWYyYzNmNjlmZThhMTc4ODFhYzIxMjAvN2Q3OGFiMzQ0MTQ2NzAwMTEyY2Q0MTYyOGFjN2NlNTRiNzljMDg2OGZlMGM3YzIwMTc1MGQ4MjM3YjU0ZGJiND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXR

In [ ]:
%cd $cpp_repo
!pip install -r requirements.txt
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM

base_model="togethercomputer/RedPajama-INCITE-Base-3B-v1"
model = AutoModelForCausalLM.from_pretrained(base_model)

for v in model.state_dict().items():
  print(v[0])

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

In [ ]:
from transformers import AutoModelForCausalLM
base_model = 'rinna/japanese-gpt-neox-3.6b-instruction-sft'
model = AutoModelForCausalLM.from_pretrained(base_model)

for v in model.state_dict().items():
  print(v[0])

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

In [ ]:
## for gpt-neox
## https://github.com/ggerganov/ggml/blob/master/examples/gpt-neox/convert-h5-to-ggml.py

%cd $cpp_repo

import sys
import struct
import json
import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer, T5Tokenizer

base_model = 'rinna/japanese-gpt-neox-3.6b-instruction-sft'
fname_out = "/content/ggml/models/gglm.bin"
model_config = "/content/ggml/models/config.json"

ftype_str = ["f32", "f16"]
ftype = 1

with open(model_config, "r", encoding="utf-8") as f:
    hparams = json.load(f)

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(base_model)


#print (model)

#print(tokenizer.encode('I believe the meaning of life is'))

list_vars = model.state_dict()
for name in list_vars.keys():
    print(name, list_vars[name].shape, list_vars[name].dtype)

fout = open(fname_out, "wb")

print(hparams)

fout.write(struct.pack("i", 0x67676d6c)) # magic: ggml in hex
fout.write(struct.pack("i", hparams["vocab_size"]))
fout.write(struct.pack("i", hparams["max_position_embeddings"]))
fout.write(struct.pack("i", hparams["hidden_size"]))
fout.write(struct.pack("i", hparams["num_attention_heads"]))
fout.write(struct.pack("i", hparams["num_hidden_layers"]))
fout.write(struct.pack("i", int(hparams["rotary_pct"]*(hparams["hidden_size"]//hparams["num_attention_heads"]))))
fout.write(struct.pack("i", hparams["use_parallel_residual"]))
fout.write(struct.pack("i", ftype))

# TODO: temporary hack to not deal with implementing the tokenizer
dot_token = tokenizer.encode('.')[0]
for i in range(hparams["vocab_size"]):
    text = tokenizer.decode([dot_token, i]).encode('utf-8')
    # remove the first byte (it's always '.')
    text = text[1:]
    fout.write(struct.pack("i", len(text)))
    fout.write(text)

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape)

    # we don't need these
    #if name.endswith(".attention.masked_bias") or     \
    #   name.endswith(".attention.bias") or \
    #   name.endswith(".attention.rotary_emb.inv_freq"):
    #    print("  Skipping variable: " + name)
    #    continue

    n_dims = len(data.shape);

    # ftype == 0 -> float32, ftype == 1 -> float16
    ftype_cur = 0;
    if ftype != 0:
        if name[-7:] == ".weight" and n_dims == 2:
            print("  Converting to float16")
            data = data.astype(np.float16)
            ftype_cur = 1
        else:
            print("  Converting to float32")
            data = data.astype(np.float32)
            ftype_cur = 0
    else:
        if data.dtype != np.float32:
            print("  Converting to float32")
            data = data.astype(np.float32)
            ftype_cur = 0

    # header
    str = name.encode('utf-8')
    fout.write(struct.pack("iii", n_dims, len(str), ftype_cur))
    for i in range(n_dims):
        fout.write(struct.pack("i", data.shape[n_dims - 1 - i]))
    fout.write(str);

    # data
    data.tofile(fout)

fout.close()

print("Done. Output file: " + fname_out)
print("")

/content/ggml


ReadTimeout: ignored

## redpajama




In [ ]:
%cd /content
!git clone https://github.com/togethercomputer/redpajama.cpp.git
%cd /content/redpajama.cpp
cpp_repo='/content/redpajama.cpp'

/content
Cloning into 'redpajama.cpp'...
remote: Enumerating objects: 2476, done.
remote: Counting objects: 100% (1298/1298), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 2476 (delta 1227), reused 1207 (delta 1207), pack-reused 1178
Receiving objects: 100% (2476/2476), 2.20 MiB | 7.54 MiB/s, done.
Resolving deltas: 100% (1638/1638), done.
/content/redpajama.cpp


In [ ]:
!mkdir -p $cpp_repo/models
%cd $cpp_repo/models

!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/config.json
#!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/tokenizer_config.json
#!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.model

/content/redpajama.cpp/models
--2023-05-18 11:58:50--  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 13.35.24.56, 13.35.24.127, 13.35.24.126, ...
Connecting to huggingface.co (huggingface.co)|13.35.24.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 534 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     534  --.-KB/s    in 0s      

2023-05-18 11:58:50 (104 MB/s) - ‘config.json’ saved [534/534]



In [ ]:
!mkdir -p $cpp_repo/models
%cd $cpp_repo/models

!wget "https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin"

/content/redpajama.cpp/models
--2023-05-18 11:33:53--  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 13.35.24.18, 13.35.24.126, 13.35.24.56, ...
Connecting to huggingface.co (huggingface.co)|13.35.24.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/e9/58e9ff5c59adafdd9012d69c25c4272a64be26c511f2c3f69fe8a17881ac2120/0c6124c628f8ecc29be1b6ee0625670062340f5b99cfe543ccf049fa90e6207b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1684665826&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2U5LzU4ZTlmZjVjNTlhZGFmZGQ5MDEyZDY5YzI1YzQyNzJhNjRiZTI2YzUxMWYyYzNmNjlmZThhMTc4ODFhYzIxMjAvMGM2MTI0YzYyOGY4ZWNjMjliZTFiNmVlMDYyNTY3MDA2MjM0MGY1Yjk5Y2ZlNTQzY2N

In [ ]:
%cd $cpp_repo

!pip install sentencepiece
!pip install transformers

/content/redpajama.cpp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## https://github.com/togethercomputer/redpajama.cpp/blob/master/examples/redpajama/scripts/convert_gptneox_to_ggml.py

#@title convert gptneox to ggml { display-mode: "form" }
#model_name =  'rinna/japanese-gpt-neox-3.6b-instruction-sft' #@param {type:"string"}
model_name =  f'{output_path}/hf_ckpt'
dir_out = f'{output_path}/ggml'

tokenizer_model = 'cyberagent/open-calm-7b'

import io
import os
import sys
import struct
import json
import code
import torch
import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer

os.makedirs(dir_out, exist_ok=True)

ftype_str = ["f32", "f16"]
ftype = 1

model_cache_dir = dir_out + "-cache"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
print("Loading model: ", model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16 if ftype == 1 else torch.float32, 
                                             cache_dir=model_cache_dir)
model.eval()
for p in model.parameters():
    p.requires_grad = False
hparams = model.config.to_dict()
print("Model loaded: ", model_name)

fn_bin = f"/ggml-{model_name.split('/')[-1]}-{ftype_str[ftype]}.bin"
fn_out = dir_out + fn_bin
fout = open(fn_out, "wb")

ggml_file_magic = 0x67676d66 # 0x67676d6c is unversioned
ggml_file_version = 0x00000001 # v1

hparams["multiple_of"] = 1
fout.write(struct.pack("i", ggml_file_magic)) # magic: ggmf in hex
fout.write(struct.pack("i", ggml_file_version))
fout.write(struct.pack("i", hparams["vocab_size"]))
fout.write(struct.pack("i", hparams["max_position_embeddings"]))
fout.write(struct.pack("i", hparams["hidden_size"]))
fout.write(struct.pack("i", hparams["num_attention_heads"]))
fout.write(struct.pack("i", hparams["num_hidden_layers"]))
fout.write(struct.pack("i", int((hparams["hidden_size"] / hparams["num_attention_heads"]
                             ) * hparams["rotary_pct"]))) # rotary_dim
fout.write(struct.pack("i", int(hparams["use_parallel_residual"])))
fout.write(struct.pack("i", ftype))

# Is this correct??
dot_token = tokenizer.encode(".")[0]
for i in range(hparams["vocab_size"]):
    text = tokenizer.decode([i]).encode('utf-8')
    fout.write(struct.pack("i", len(text)))
    fout.write(text)

list_vars = model.state_dict()

print(hparams)

for name in list_vars.keys():
    if name.startswith('gpt_neox.layers.'):
        if 'attention.masked_bias' in name or \
            'attention.rotary_emb.inv_freq' in name or \
            'attention.bias' in name:
            continue
    # No gradients for these
    list_vars[name].requires_grad = False
    src = name
    nn = name

    print(src, ' -> ', name)
    data = list_vars[src].squeeze().numpy()
    data = data.astype(np.float32)

    n_dims = len(data.shape)
    print(name, n_dims, data.shape)

    # default type is fp32
    ftype_cur = 0
    if ftype == 1 and n_dims > 1:
        print("  Converting to float16", data.shape, data[:3, :3].tolist())
        data = data.astype(np.float16)
        ftype_cur = 1
    else:
        print("  Converting to float32", data.shape,
              data[:3, :3].tolist() if n_dims > 1 else data[:3].tolist())
        data = data.astype(np.float32)

    # header
    str = name.encode('utf-8')
    fout.write(struct.pack("iii", n_dims, len(str), ftype_cur))
    for i in range(n_dims):
        fout.write(struct.pack("i", data.shape[n_dims - 1 - i]))
    print(str)
    fout.write(str)

    # data
    data.tofile(fout)

fout.close()

print("Done. Output file: " + fn_out)
print("")

Loading model:  /content/drive/MyDrive/models/calm_dolly_7B/hf_ckpt


Loading checkpoint shards:   0%|          | 0/45 [00:00<?, ?it/s]

Model loaded:  /content/drive/MyDrive/models/calm_dolly_7B/hf_ckpt
{'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'float16', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': False, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty': 0.0, 'temperature': 1.0, 'top_k': 50, 'top_p': 1.0, 'typical_p': 1.0, 'repetition_penalty': 1.0, 'length_penalty': 1.0, 'no_repeat_ngram_size': 0, 'encoder_no_repeat_ngram_size': 0, 'bad_words_ids': None, 'num_return_sequences': 1, 'chunk_size_feed_forward': 0, 'output_scores': False, 'return_dict_in_generate': False, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'remove_invalid_values': False, 'exponenti

In [ ]:
%cd $cpp_repo
!make quantize-gptneox redpajama

/content/redpajama.cpp
I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

make: 'quantize-gptneox' is up to date.
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -c examples/redpajama/common-gptneox.cpp -o common-gptneox.o
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native 

In [ ]:
#@title quantize { display-mode: "form" }
# model_name =  '/content/redpajama.cpp/models/ggml-japanese-gpt-neox-3.6b-instruction-sft-f16.bin' #@param {type:"string"}
model_name =  f'{output_path}/ggml/ggml-hf_ckpt-f16.bin'

%cd $cpp_repo

!python ./examples/redpajama/scripts/quantize-gptneox.py $model_name --quantize-output-type q8_0

/content/redpajama.cpp
gptneox.cpp: loading model from /content/drive/MyDrive/models/calm_dolly_7B/ggml/ggml-hf_ckpt-f16.bin
gptneox.cpp: saving model to /content/drive/MyDrive/models/calm_dolly_7B/ggml/ggml-hf_ckpt-q8_0.bin
[   1/ 388]             gpt_neox.embed_in.weight -     4096 x 52224, type =    f16, quantizing .. size =   408.00 MiB ->   229.50 MiB | hist: 0.000 0.027 0.020 0.031 0.047 0.067 0.088 0.106 0.226 0.106 0.088 0.067 0.047 0.031 0.020 0.027 
[   2/ 388] gpt_neox.layers.0.input_layernorm.weight -             4096, type =    f32, size =    0.016 MiB
[   3/ 388] gpt_neox.layers.0.input_layernorm.bias -             4096, type =    f32, size =    0.016 MiB
[   4/ 388] gpt_neox.layers.0.post_attention_layernorm.weight -             4096, type =    f32, size =    0.016 MiB
[   5/ 388] gpt_neox.layers.0.post_attention_layernorm.bias -             4096, type =    f32, size =    0.016 MiB
[   6/ 388] gpt_neox.layers.0.attention.query_key_value.weight -     4096 x 12288, type = 

In [ ]:
src="/content/redpajama.cpp/models/ggml-japanese-gpt-neox-3.6b-instruction-sft-q8_0.bin"
target="/content/drive/MyDrive/models/rinna_3B_cpp/ggml-rinna-3B-q8_0.bin"
!cp $src $target

In [ ]:
#@title infarence ggml { display-mode: "form" }
#model =  'rinna/japanese-gpt-neox-3.6b-instruction-sft' #@param {type:"string"}
model = f'{output_path}/ggml/ggml-hf_ckpt-q8_0.bin'

%cd $cpp_repo

prompt = [
    {
        "speaker": "ユーザー",
        "text": "日本のおすすめの観光地を教えてください。"
    },
    {
        "speaker": "システム",
        "text": "どの地域の観光地が知りたいですか？"
    },
    {
        "speaker": "ユーザー",
        "text": "渋谷の観光地を教えてください。"
    }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)

instruction = "日本で1番高い山を教えてください。"
prompt=f"""
以下は、ある作業を記述した指示です。依頼を適切に完了させる応答を書きなさい

### 指示:
{instruction}

### 応答:
"""

!./redpajama -m "$model" \
  -c 1024 \
  -b 128 \
  -n 1 \
  -t 8 \
  --color \
  --top_k 30 \
  --top_p 0.95 \
  --temp 0.2 \
  --repeat_last_n 3 \
  --repeat_penalty 1.1 \
  --seed 0 \
  --n_predict 256 \
  --verbose-prompt \
  -p "$prompt"

/content/redpajama.cpp
main: seed = 0
gptneox.cpp: loading model from /content/drive/MyDrive/models/calm_dolly_7B/ggml/ggml-hf_ckpt-q8_0.bin
gptneox_model_load_internal: format     = ggjt v1 (latest)
gptneox_model_load_internal: n_vocab    = 52224
gptneox_model_load_internal: n_ctx      = 1024
gptneox_model_load_internal: n_embd     = 4096
gptneox_model_load_internal: n_head     = 32
gptneox_model_load_internal: n_layer    = 32
gptneox_model_load_internal: n_rot      = 128
gptneox_model_load_internal: use_parallel_residual = 0
gptneox_model_load_internal: ftype      = 7 (mostly Q8_0)
gptneox_model_load_internal: n_parts    = 1
gptneox_model_load_internal: model size = 7B
gptneox_model_load_internal: ggml ctx size =  90.94 KiB
gptneox_model_load_internal: mem required  = 9169.62 MiB (+ 1026.00 MiB per state)
gptneox_init_from_file: kv self size  =  512.00 MiB

system_info: n_threads = 8 / 4 | AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_

### cpp 2 (fail)

In [ ]:
%cd /content
!git clone https://github.com/if001/llama.cpp.git
cpp_repo="/content/llama.cpp"

/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 2579, done.
remote: Counting objects: 100% (1121/1121), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 2579 (delta 1024), reused 954 (delta 938), pack-reused 1458
Receiving objects: 100% (2579/2579), 2.44 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (1685/1685), done.


In [ ]:
%cd $cpp_repo
!mkdir -p build
%cd $cpp_repo/build
!cmake ..
!cmake --build . --config Release

/content/llama.cpp
/content/llama.cpp/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.25.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Configuring done
-- Generating done
-- Build files have been written to: /content/llama.cpp/build
[  3%] Built target BUILD_INFO
[  6%] Building 

In [ ]:
!mkdir -p $cpp_repo/models/3B
%cd $cpp_repo/models/3B

!wget "https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin"

%cd $cpp_repo/models

!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.model
!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/spiece.vocab
!wget https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/config.json

/content/llama.cpp/models/3B
--2023-05-17 15:23:32--  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 13.35.24.56, 13.35.24.126, 13.35.24.18, ...
Connecting to huggingface.co (huggingface.co)|13.35.24.56|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/e9/58e9ff5c59adafdd9012d69c25c4272a64be26c511f2c3f69fe8a17881ac2120/0c6124c628f8ecc29be1b6ee0625670062340f5b99cfe543ccf049fa90e6207b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1684596222&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU4L2U5LzU4ZTlmZjVjNTlhZGFmZGQ5MDEyZDY5YzI1YzQyNzJhNjRiZTI2YzUxMWYyYzNmNjlmZThhMTc4ODFhYzIxMjAvMGM2MTI0YzYyOGY4ZWNjMjliZTFiNmVlMDYyNTY3MDA2MjM0MGY1Yjk5Y2ZlNTQzY2Nm

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00


In [ ]:
%cd $cpp_repo/models
!ls


/content/llama.cpp/models
3B  config.json  ggml-vocab.bin  spiece.model  spiece.vocab


In [ ]:
%cd $cpp_repo

!python3 convert.py models/3B/ --vocab-dir="/content/llama.cpp/models/spiece.model"

/content/llama.cpp
Loading model file models/3B/pytorch_model.bin
Loading vocab file /content/llama.cpp/models/spiece.model
Traceback (most recent call last):
  File "/content/llama.cpp/convert.py", line 1171, in <module>
    main()
  File "/content/llama.cpp/convert.py", line 1162, in main
    output_type = pick_output_type(model, args.outtype)
  File "/content/llama.cpp/convert.py", line 981, in pick_output_type
    wq_type = model["layers.0.attention.wq.weight"].data_type
KeyError: 'layers.0.attention.wq.weight'


## lora merge

In [ ]:
!pip install git+https://github.com/huggingface/peft.git transformers sentencepiece torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-mvkw28jf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-mvkw28jf
  Resolved https://github.com/huggingface/peft.git to commit 3714aa2fff158fdfa637b2b65952580801d890b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer  # noqa: E402

model_name = 'cyberagent/open-calm-7b'
lora_weights = output_path

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map={"": "cpu"},
)

#first_weight = base_model.model.gpt_neox.layers[0].attention.query_key_value.weight
#first_weight_old = first_weight.clone()

lora_model = PeftModel.from_pretrained(
    base_model,
    lora_weights,
    device_map={"": "cpu"},
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# merge weights
# print(base_model.config)

#for layer in lora_model.base_model.model.gpt_neox.layers:
#    layer.attention.query_key_value.merge_weights = True
  
lora_model_sd = lora_model.state_dict()
for k, v in lora_model_sd.items():
  print(k)

base_model.model.gpt_neox.embed_in.weight
base_model.model.gpt_neox.layers.0.input_layernorm.weight
base_model.model.gpt_neox.layers.0.input_layernorm.bias
base_model.model.gpt_neox.layers.0.post_attention_layernorm.weight
base_model.model.gpt_neox.layers.0.post_attention_layernorm.bias
base_model.model.gpt_neox.layers.0.attention.bias
base_model.model.gpt_neox.layers.0.attention.masked_bias
base_model.model.gpt_neox.layers.0.attention.rotary_emb.inv_freq
base_model.model.gpt_neox.layers.0.attention.query_key_value.weight
base_model.model.gpt_neox.layers.0.attention.query_key_value.bias
base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_A.default.weight
base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_B.default.weight
base_model.model.gpt_neox.layers.0.attention.dense.weight
base_model.model.gpt_neox.layers.0.attention.dense.bias
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.weight
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.bias
base_mod

In [ ]:
%cd $output_path

import json
import os
from transformers import GPTNeoXForCausalLM

#for layer in lora_model.base_model.model.gpt_neox.layers:
#    layer.attention.query_key_value.merge_weights = True


# merge weights - new merging method from peft
merged_model = lora_model.merge_and_unload()

merged_model.train(False)

# did we do anything?
# assert not torch.allclose(first_weight_old, first_weight)

lora_model_sd = merged_model.state_dict()
deloreanized_sd = {
    k.replace("base_model.model.", ""): v
    for k, v in lora_model_sd.items()
    if "lora" not in k
}

os.makedirs(f'{output_path}/hf_ckpt', exist_ok=True)
GPTNeoXForCausalLM.save_pretrained(
    base_model,
    save_directory=f"{output_path}/hf_ckpt", state_dict=deloreanized_sd, max_shard_size="400MB"
)

/content/drive/MyDrive/models/calm_dolly_7B
